# Demo for genus 3

Demo for [this presentation](http://page.math.tu-berlin.de/~joswig/presentations/Joswig-DMG-201209.pdf).
The setup defines many global variables.

In [1]:
script "g3/setup.pl";

For instance, the genus.

In [2]:
print $genus;

3

The point configuration is fixed.  These are the 15 lattice points of T4.  Later on we will use their indices; hence that particular ordering will be kept throughout.

In [3]:
print rows_labeled($points);

0:1 0 0
1:1 1 0
2:1 0 1
3:1 2 0
4:1 1 1
5:1 0 2
6:1 3 0
7:1 2 1
8:1 1 2
9:1 0 3
10:1 4 0
11:1 3 1
12:1 2 2
13:1 1 3
14:1 0 4


The first part of the computation comes from TOPCOM.  This is where the output resides.

In [4]:
print $triangulations_file;

g3/preprocessing/g3TriangulationsData/g3FineRegularAffine.txt

Here is an example how to read all 1278 regular and unimodular triangulations of T4 (up to symmetry) into one perl array.

In [5]:
@T=();
open (TOPCOM, "<:encoding(UTF-8)", "$triangulations_file");
foreach (<TOPCOM>) { chomp; push @T, new Array<Set>(eval($_)); }
close TOPCOM;

These triangulations are referenced by their index, which is the line number in the TOPCOM output, starting at zero.  For the rest we will look at the triangulation which happens to come first.

In [6]:
print scalar(@T), "\n", $T[0];

1278
{0 1 2}
{1 2 3}
{2 3 4}
{2 4 5}
{3 4 6}
{4 5 6}
{5 6 7}
{5 7 8}
{5 8 9}
{6 7 10}
{7 8 10}
{8 9 10}
{9 10 11}
{9 11 12}
{9 12 13}
{9 13 14}


Our object of interest is the (closed) secondary cone of that unimodular triangulation.

In [7]:
$subdiv = new fan::SubdivisionOfPoints(POINTS=>$points,MAXIMAL_CELLS=>$T[0]);
$sec = $subdiv->secondary_cone;        

And these are the rays.

In [8]:
print $sec->RAYS;

0 0 0 1 1 1 2 2 2 2 3 3 3 3 3
0 0 0 0 1 2 1 2 3 4 2 3 4 5 6
0 0 0 0 0 0 0 1 2 3 2 3 4 5 6
0 0 0 0 0 0 0 0 1 2 0 1 2 3 4
0 0 0 0 0 0 0 0 0 0 0 1 2 3 4
0 0 0 0 0 0 0 0 0 0 0 0 1 2 3
0 0 0 0 0 0 0 0 0 0 0 0 0 1 2
0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
0 0 0 0 0 0 0 0 0 1 0 1 2 3 4
0 0 0 0 0 0 0 0 0 0 1 1 1 1 1
0 0 0 0 0 1 0 1 2 3 1 2 3 4 5
0 0 0 0 0 0 1 1 1 1 2 2 2 2 2


Click here for additional output 
 
polymake: used package ppl
 The Parma Polyhedra Library ([[wiki:external_software#PPL]]): A C++ library for convex polyhedra
 and other numerical abstractions.
 http://www.cs.unipr.it/ppl/

It is not necessary to do this computation.  The data set contains that secondary cone as apolymake object, with a number of useful properties pre-computed.  No need to do this again. 

In [9]:
$sec0 = load("g3/secondary_fan/g3b0/g3b0-0.cone");
print $sec0->$_, " " for "DIM", "AMBIENT_DIM";

12 15 

That's a polymake `BigObject` of type `Cone` with rational coordinates.  It can be used like any other cone in polymake.

In [10]:
print $sec0->type->full_name;

Cone<Rational>

However, these cones contain additional information, which is stored as "attachments".  For instance, the secondary cone remembers the triangulation where it came from.  Redundant information like this is extremely important to follow the computational path or to spot any errors.

In [11]:
print $sec0->get_attachment("INDUCED_TRIANGULATION");

{0 1 2}
{1 2 3}
{2 3 4}
{2 4 5}
{3 4 6}
{4 5 6}
{5 6 7}
{5 7 8}
{5 8 9}
{6 7 10}
{7 8 10}
{8 9 10}
{9 10 11}
{9 11 12}
{9 12 13}
{9 13 14}


The main computation is to map each secondary cone via some linear map into the moduli space.  The image is the corresponding moduli cone.  The naming scheme is derived from the name of the secondary cone.  This makes it easy to find matching cones.

In [12]:
$mod0 = load("g3/secondary_fan/g3b0/moduli/g3b0-0-moduli.cone");
print $mod0->$_, " " for "DIM", "AMBIENT_DIM";

5 6 

Again the moduli cones have several attachments.

In [13]:
print join " ", $mod0->list_attachments();

GENUS POLYTOPE ID ZERO_ROW_IDX BUCKET

For instance, a moduli cone knows its id (= id of its triangulation).  This way accidentally renaming files does not destroy important information.  This is another example of redundant information which is put in everywhere.

In [14]:
print $mod0->get_attachment("ID");

0

Geometrically relevant are the rays of the moduli cone.

In [15]:
print $mod0->RAYS;

0 0 0 0 1 0
0 0 1 1 0 0
1 0 0 0 0 0
1 1 1 0 0 0
0 0 0 0 0 1


Yet this does not reveal how the linear map from the secondary cone works.  To this end, let us look at how that moduli cone was "born".  The property `INPUT_RAYS` may have redundant generators (in contrast to `RAYS`).  here, e.g., you can see that the last two vectors are the same.

In [16]:
print rows_numbered($mod0->INPUT_RAYS);

0:0 0 0 0 1 0
1:0 0 1 1 0 0
2:1 0 0 0 0 0
3:1 1 1 0 0 0
4:0 0 0 0 0 1
5:0 0 0 0 0 1


To understand the map from the secondary cone to its moduli cone, the attachment `ZERO_ROW_IDX` is important.  It tells which rays of the secondary cone are mapped to zero.  It is necessary to store this extra, as it is illegal to have zero vectors in `INPUT_RAYS`.

In [17]:
print $mod0->get_attachment("ZERO_ROW_IDX");

0, 1, 4, 5, 6, 7

The secondary cone has a 3-dimensional lineality space, which allows for distinct choices of rays.  Hence, in order to define the map to the moduli space correctly, it is important to pick the rays consistently.  These are the `STANDARD_RAYS`, which again are stored as an attachment.  In this case they are made unique by (1) mapping the three vertices of T4 to zero and (2) picking the unique primitive vector in the one-dimensional subspace defined by a ray.

In [18]:
print rows_numbered($sec0->get_attachment("STANDARD_RAYS"));

0:1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
1:0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
2:0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
3:0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0
4:0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
5:0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
6:0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
7:0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
8:0 0 0 0 0 0 0 0 1 0 0 0 2 1 0 0 0 0
9:0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0
10:0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0
11:0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0


This can then be used for computationally proving Theorem 5.1 of [BJMS2015].

In [19]:
script "g3/proofs.pl";

The proof proceeds bucket by bucket, and he we are checking the bucket of the triangulation with is 0.

In [20]:
check_bucket(0);

bucket 0 of size 225:
unverified cones=0 out of 225 checked


This means that all cones satisfy the criterion the given inequalities (which were previously found by hand).